In [33]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
%matplotlib inline

HW6 和 HW7 均是讲GRS test，一个是单因子，一个是多因子，这里一并讲完。

我们说CAPM的要求是，beta可以解释所有的超额回报(这里的超额回报指的是超过无风险回报的回报)，也就是在跑下面的模型的时候，$\alpha$ 应该是等于0.

$$r_i^e = \alpha_i + \beta_i r_m^e + \epsilon_i$$

为了和之前CAPM的test区别开来，强调一下这里面的自变量是$r_m^e$，系数是$\beta_i$, 而CAPM那一章的两个test，均值$\beta$作为自变量，$r_m^e$做为系数。  

如果是最开始学的t-test， 也就是individual testing。我们的假设是对于特定的一个资产，可以是一只股票，或者是一只基金，他的abnormal return 也就是 alpha是等于0， 即：

$$H_0: \alpha_i = 0$$  
  
无论对多少个资产做的t-test,都是单独的test。举个不是特别贴切但是比较直观的例子，如果我们取5%的显著程度，你可以理解成是5%的容错度，或者说，我们有95%的几率我们对某个资产是否符合CAPM的判断是正确的。假设资产间的判断互不干扰，那么对于两个资产，如果要对他们的判断都是正确的，那么概率就是95%*95%=90%, 而这样，我们的容错度已经变成了10%.随着越多的资产加入，我们估计的错误就会越大。  

为什么要所有资产判断都是正确的？如果所有的资产都是95%的置信程度下不拒绝原假设，其中有一个实际上是要拒绝的，或者说不符合CAPM的，那么CAPM就是错的了，因为CAPM要求的是Beta解释一切回报（#人啊，说话不要说那么绝对啊）  

因此，我们引入join-test。对于N个资产，join-test的原假设是：

$$H_0: \alpha_1=\alpha_2=...=\alpha_n=0$$

In [34]:
#import portfolios returns
url1 = "https://www.dropbox.com/s/hez02fvhxymvaqi/Homework6.xls?dl=1"
changedate  = lambda x: pd.to_datetime(x,format="%Y%m")
Industry30  = pd.read_excel(url1,header=2,sheetname='Industry portfolios (VW)',    index_col=0, parse_dates=True, date_parser=changedate)/100

In [35]:
#import factors
url2 = "https://www.dropbox.com/s/9346pp2iu5prv8s/MonthlyFactors.csv?dl=1"
Factors = pd.read_csv(url2,header=0,index_col=0, parse_dates=True, na_values=-99)/100

#Market Permium
Factors.MKT = Factors.MKT.subtract(Factors.RF, axis=0)

#change index
Factors.index = Factors.index - pd.tseries.offsets.MonthBegin()

In [36]:
#Merge Datas
Ind30  = pd.merge(Factors.iloc[:,:4],Industry30,  'inner',left_index=True,right_index=True)
#Convert Return to Premium
Ind30.iloc[:,-30:]  = Ind30.iloc[:,-30:].subtract(Ind30.RF, axis=0)

Ind30.head()

,RF,MKT,SMB,HML,Food,Beer,Smoke,Games,Books,Hshld,...,Telcm,Servs,BusEq,Paper,Trans,Whlsl,Rtail,Meals,Fin,Other
1926-07-01,0.0022,0.0296,-0.0230,-0.0287,0.0034,-0.0541,0.0107,0.0271,0.1075,-0.0070,...,0.0061,0.0900,0.0184,0.0748,0.0171,-0.2401,-0.0015,0.0165,0.0015,0.0498
1926-08-01,0.0025,0.0264,-0.0140,0.0419,0.0234,0.2678,0.0625,0.0030,0.0976,-0.0383,...,0.0192,0.0177,0.0414,-0.0263,0.0463,0.0514,-0.0100,-0.0038,0.0421,0.0651
1926-09-01,0.0023,0.0036,-0.0132,0.0001,0.0093,0.0379,0.0103,0.0635,-0.0122,0.0050,...,0.0218,0.0202,-0.0004,-0.0577,-0.0018,-0.0810,0.0002,-0.0079,-0.0146,-0.0409
1926-10-01,0.0032,-0.0324,0.0004,0.0051,-0.0338,-0.0363,0.0074,-0.0508,0.0915,-0.0500,...,-0.0043,-0.0232,-0.0141,-0.0540,-0.0296,-0.1570,-0.0252,-0.0443,-0.0548,-0.0881
1926-11-01,0.0031,0.0253,-0.0020,-0.0035,0.0604,0.0698,0.0424,0.0135,-0.0611,-0.0085,...,0.0132,0.0346,0.0333,0.0353,0.0129,0.0436,0.0621,0.0402,0.0193,0.0369


GRS test：首先从最简单的单因子模型开始：

$$r_i = \alpha_i + \beta_i r_{fa} + \epsilon_i$$

注意，这个地方我没有写$r_m$, 而是写了$r_fa$. $r_fa$指一个factor的超额回报。实际上，GRS test不仅仅用于CAPM，对于任何一个factor，我们都可以用来验证，这个factor能否解释所有的超额回报，话句话说，就是用这个factor解释return的时候，所有的alpha是不是都等于0.同时还有一个特别重要的原因，下面会提到！

$$H_0: \alpha_1, \alpha_2, \alpha_3...~are~all~equal~to~0$$

针对这个假设，我们有一个chi-square统计量。

这里方便一下没有任何统计基础的同学，我讲一下。对于任何一个假设检验，我们都有一个统计量来衡量零假设是否合理
在t-test里面，我们的统计量是t，我们知道t遵循的是近似于standard normal distribution(实际上是t-distribution)
在GRS test里面，我们的统计量是chi2，这个统计量遵循的是F-distribution

F-distribution和t-distribution实际上都是和sample size相关的分布，不同sample size会有不同的分布。对于t-distribution，当sample size大于50的时候，就近似于standard norma distribution，所以我们在计算t值的时候，基本上用的都是正态分布。F-distribution就不一样了，在计算F-distribution的时候，必须引入这个变量。

chi2的计算是(每个变量我会解释)
$$chi2 = T * \frac{\alpha @ \Sigma^{-1} @ \alpha} {1+SP_{fa}^2}$$

综上所述，对于一个GRS test，我们需要的变量有：  
N: 资产量  
T: 时间量  
对于20个资产在一年间的月回报，我们会有20*12个数据，其中，20是我们的资产量，也就是N；12是我们的时间量，也就是T。  
$\alpha$: N个资产的alpha组成的列矩阵  
$\Sigma$: N个资产的epsilon的协方差矩阵  
$SP_{fa}$: 注意，这里指的是factor的shape ratio而不是market的sharp ratio。如果你是在检验CAPM，那么这里的的确确用的是market的sharp ratio。而这里如果你检验的是mve或者其它奇奇怪怪的factor，那么这里的SP_f也要换成对应的sharp ratio。

对于chi1的计算，分子是矩阵运算，分母就是数字计算。

所以，对于一个data的GRS test，我们需要有5个变量，数字变量N，T和$SP_{fa}$，大小为$(N,)$的行矩阵$\alpha$，以及大小为$(N,N)$的矩阵$\Sigma$

导入数据后，  
第一步，开变量准备记录数据。

In [37]:
#Initiate Variables
Alpha30     = pd.DataFrame( Ind30.iloc[:,-30:].mean()*0,columns = ['Market'] )
Epsilon30_c = Ind30.iloc[:,-30:]*0

第二步，分资产，做回归，我们先来做单factor，也就是market premium(CAPM)。

In [38]:
x = sm.add_constant(Ind30.MKT)
Y = Ind30.iloc[:,4:]
for industry in Alpha30.index:
    y = Y[industry]                                          #pick one industry
    results = sm.OLS(y,x).fit()
    Alpha30.set_value(industry, 'Market', results.params[0]) #record alpha
    Epsilon30_c[industry] = results.resid                    #record residuals    

第三步，准备五个值

In [39]:
N = Y.shape[1]
T = Y.shape[0]
SP_m = Ind30.MKT.mean()/Ind30.MKT.std() 
alpha = Alpha30.Market.values                     #转换，不然无法做矩阵运算
InvSigma = np.linalg.inv(Epsilon30_c.cov())

第四步，算chi2，求p-value

In [40]:
chi2 = T * (alpha.T @ InvSigma @ alpha) / (1 + SP_m**2)
print("The F-statistic of GRS Test is: " + str(round(chi2,4)))
print("The p-value of F-statistic is: " + str(round(1-stats.chi2.cdf(chi2,N),4)))

The F-statistic of GRS Test is: 59.1674
The p-value of F-statistic is: 0.0012


如果是three factors model，那么SP2_fa就有另外的公式了：  

$$SP^2_{fa}=E(F)^TSigma_F^{-1}E(F)$$

这个公式是个什么鬼？其实很简单，这个时候我们的回报模型已经变成了(以Three Factors为例)  

$$r_i = \alpha_i + \beta_{i,1} r_{f1} + \beta_{i,2} r_{f2} + \beta_{i,3} r_{f3} + \epsilon_i$$

如果我们把所有的factor组合在一起，成为一个更加牛逼的portfolio_fa:

$$r_{fa} = w_1 r_{f1} + w_2 r_{f2} + w_3 r_{f3}$$

$$r_i = \alpha_i + \beta_{i,fa} r_{fa} + \epsilon_i$$

如同CAPM里面，market portfolio应该是tangency portfolio，在这里面，这个新的portfolio应该也是含有三个因子的tangency portfolio，所以，根据我在portfolio theory note里面的公式，它的Sharp Ratio应该是：

$$SP_{fa} = \sqrt{E(F)^T@ \Sigma_F^{-1} @ E(F)}$$

而现在，我们成功的把three factors model 转换成 one factor model，那么$SP^2_{fa}$只要做相应变换就好了。

接下来我们做三因子的GRS test

In [41]:
#Initiate Variables
Alpha30['ThreeFactors'] = Alpha30['Market']*0
Epsilon30_t = Ind30.iloc[:,-30:]*0

In [42]:
Factors=Ind30.iloc[:,1:4]                                    #pick three factors
x = sm.add_constant(Factors)                       
Y = Ind30.iloc[:,4:]
for industry in Alpha30.index:
    y = Y[industry]                                          #pick one industry
    results = sm.OLS(y,x).fit()
    Alpha30.set_value(industry, 'ThreeFactors', results.params[0]) #record alpha
    Epsilon30_t[industry] = results.resid                    #record residuals 

In [43]:
N = Y.shape[1]
T = Y.shape[0]
SP_f  = Factors.mean().values.T @ np.linalg.inv(Factors.cov()) @ Factors.mean().values
alpha = Alpha30.ThreeFactors.values                     
InvSigma = np.linalg.inv(Epsilon30_t.cov())

In [44]:
chi2 = T * (alpha.T @ InvSigma @ alpha) / (1 + SP_f**2)
print("The F-statistic of GRS Test is: " + str(round(chi2,4)))
print("The p-value of F-statistic is: " + str(round(1-stats.chi2.cdf(chi2,N),4)))

The F-statistic of GRS Test is: 96.1222
The p-value of F-statistic is: 0.0


我们来深入研究一下 $\alpha$ 和 $\epsilon$ 的关系，以下是我的理解，不一定正确。  
$\alpha_i$是某个资产的abnormal return的期望，而$\epsilon_i$即是$\alpha$的波动。理解这一点，$r_i-\beta_ir_m=\alpha_i+\epsilon_i$是某个资产的abnormal return，而且因为$\alpha_i$是常数,  

$$E(r_i-\beta_ir_m)=E(\alpha_i)+E(\epsilon_i)=\alpha_i+0$$  

$$Var(r_i-\beta_ir_m)=Var(\alpha_i+\epsilon_i)=Var(\epsilon_i)$$

再来看看Appraisal ratio  

$$Appraisal=\alpha/\sigma_\epsilon$$

有没有觉得特别熟悉？没错，如果我们定义hedging portfolio： 

$$r^{hedged}_i=r^e_i−\beta_ir_m$$

实际上，Appraisal Ratio 就是 $r^{hedged}_i$ 的 Shape Ratio